In [24]:
import openmeteo_requests
import pandas as pd

import requests_cache
import pandas as pd
from retry_requests import retry

In [25]:
# Read in locations 
locations = pd.read_csv('../../../locations/locations.txt', sep=',', header=0)
# Convert columns lat and lon to list
lat = locations['lat'].tolist()
lon = locations['lon'].tolist()

In [26]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": lat,
	"longitude": lon,
	"start_date": "2000-01-01",
	"end_date": "2024-04-30",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "snowfall", "surface_pressure", "et0_fao_evapotranspiration", "wind_direction_10m", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "shortwave_radiation"],
	"models": "best_match", #  ECMWF IFS & ERA5 (Combines ERA5 and ERA5-Land seamlessly)
    "cell_slection": "land", # Land finds a suitable grid-cell on land with similar elev. to the requested coord. using a 90-meter DEM
 }
responses = openmeteo.weather_api(url, params=params)

In [27]:
# Initialize a dictionary to store DataFrames for each station
dataframes = {}

# Loop over each response and process data for each station
for i, response in enumerate(responses):
    station_key = f"id_{i+1}"
    print(f"Coordinates {response.Latitude()}\u00b0N {response.Longitude()}\u00b0E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
    hourly_rain = hourly.Variables(4).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(5).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(6).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(7).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(8).ValuesAsNumpy()
    hourly_soil_moisture_0_to_7cm = hourly.Variables(9).ValuesAsNumpy()
    hourly_soil_moisture_7_to_28cm = hourly.Variables(10).ValuesAsNumpy()
    hourly_soil_moisture_28_to_100cm = hourly.Variables(11).ValuesAsNumpy()
    hourly_shortwave_radiation = hourly.Variables(12).ValuesAsNumpy()

    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )
    }
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
    hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
    hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation

    # Create a DataFrame for the current station
    hourly_dataframe = pd.DataFrame(data=hourly_data)
    dataframes[station_key] = hourly_dataframe

    # Print the DataFrame for the current station (optional)
    print(hourly_dataframe)

Coordinates 51.9156379699707°N 10.219780921936035°E
Elevation 275.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                            date  temperature_2m  relative_humidity_2m  \
0      2000-01-01 00:00:00+00:00       -1.045500             94.994484   
1      2000-01-01 01:00:00+00:00       -1.095500             95.342621   
2      2000-01-01 02:00:00+00:00       -0.995500             95.346375   
3      2000-01-01 03:00:00+00:00       -0.795500             96.055969   
4      2000-01-01 04:00:00+00:00       -0.795500             96.762627   
...                          ...             ...                   ...   
213283 2024-04-30 19:00:00+00:00       16.629999             86.852814   
213284 2024-04-30 20:00:00+00:00       15.730000             85.923889   
213285 2024-04-30 21:00:00+00:00       16.029999             76.401001   
213286 2024-04-30 22:00:00+00:00       16.179998             71.774422   
213287 2024-04-30 23:00:00+00:00       15.580000            

In [28]:
# save all dataframes to a seperate csv files in the data folder
for key, df in dataframes.items():
    df.to_csv(f"../station_{key}_hourly_2000_2024.csv", index=False)